# gensim doc2vec & IMDB sentiment dataset

TODO: section on introduction & motivation

TODO: prerequisites + dependencies (statsmodels, patsy, ?)

## Load corpus

Fetch and prep exactly as in Mikolov's go.sh shell script. (Note this cell tests for existence of required files, so steps won't repeat once the final summary file (`aclImdb/alldata-id.txt`) is available alongside this notebook.)

In [1]:
# %%bash
# # adapted from Mikolov's example go.sh script: 
# if [ ! -f "aclImdb/alldata-id.txt" ]
# then
#     if [ ! -d "aclImdb" ] 
#     then
#         if [ ! -f "aclImdb_v1.tar.gz" ]
#         then
#           wget --quiet http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#         fi
#       tar xf aclImdb_v1.tar.gz
#     fi
    
#   #this function will convert text to lowercase and will disconnect punctuation and special symbols from words
#   function normalize_text {
#     awk '{print tolower($0);}' < $1 | sed -e 's/\./ \. /g' -e 's/<br \/>/ /g' -e 's/"/ " /g' \
#     -e 's/,/ , /g' -e 's/(/ ( /g' -e 's/)/ ) /g' -e 's/\!/ \! /g' -e 's/\?/ \? /g' \
#     -e 's/\;/ \; /g' -e 's/\:/ \: /g' > $1-norm
#   }

#   export LC_ALL=C
#   for j in train/pos train/neg test/pos test/neg train/unsup; do
#     rm temp
#     for i in `ls aclImdb/$j`; do cat aclImdb/$j/$i >> temp; awk 'BEGIN{print;}' >> temp; done
#     normalize_text temp
#     mv temp-norm aclImdb/$j/norm.txt
#   done
#   mv aclImdb/train/pos/norm.txt aclImdb/train-pos.txt
#   mv aclImdb/train/neg/norm.txt aclImdb/train-neg.txt
#   mv aclImdb/test/pos/norm.txt aclImdb/test-pos.txt
#   mv aclImdb/test/neg/norm.txt aclImdb/test-neg.txt
#   mv aclImdb/train/unsup/norm.txt aclImdb/train-unsup.txt

#   cat aclImdb/train-pos.txt aclImdb/train-neg.txt aclImdb/test-pos.txt aclImdb/test-neg.txt aclImdb/train-unsup.txt > aclImdb/alldata.txt
#   awk 'BEGIN{a=0;}{print "_*" a " " $0; a++;}' < aclImdb/alldata.txt > aclImdb/alldata-id.txt
# fi

In [2]:
import os.path
assert os.path.isfile("aclImdb/alldata-id.txt"), "alldata-id.txt unavailable"

The data is small enough to be read into memory. 

In [1]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order
with open('aclImdb/alldata-id.txt') as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split()
        words = tokens[1:]
        tags = [line_no] # `tags = [tokens[0]]` would also work at extra memory cost
        split = ['train','test','extra','extra'][line_no//25000]  # 25k train, 25k test, 25k extra
        sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//12500] # [12.5K pos, 12.5K neg]*2 then unknown
        alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

unable to import 'smart_open.gcs', disabling that module


100000 docs: 25000 train-sentiment, 25000 test-sentiment


## Set-up Doc2Vec Training & Evaluation Models

Approximating experiment of Le & Mikolov ["Distributed Representations of Sentences and Documents"](http://cs.stanford.edu/~quocle/paragraph_vector.pdf), also with guidance from Mikolov's [example go.sh](https://groups.google.com/d/msg/word2vec-toolkit/Q49FIrNOQRo/J6KG8mUj45sJ):

`./word2vec -train ../alldata-id.txt -output vectors.txt -cbow 0 -size 100 -window 10 -negative 5 -hs 0 -sample 1e-4 -threads 40 -binary 0 -iter 20 -min-count 1 -sentence-vectors 1`

Parameter choices below vary:

* 100-dimensional vectors, as the 400d vectors of the paper don't seem to offer much benefit on this task
* similarly, frequent word subsampling seems to decrease sentiment-prediction accuracy, so it's left out
* `cbow=0` means skip-gram which is equivalent to the paper's 'PV-DBOW' mode, matched in gensim with `dm=0`
* added to that DBOW model are two DM models, one which averages context vectors (`dm_mean`) and one which concatenates them (`dm_concat`, resulting in a much larger, slower, more data-hungry model)
* a `min_count=2` saves quite a bit of model memory, discarding only words that appear in a single doc (and are thus no more expressive than the unique-to-each doc vectors themselves)

In [2]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
print(cores)
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

# speed setup by sharing results of 1st model's vocabulary scan
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
print(simple_models[0])
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

12


/usr/local/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)
Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)


Following the paper, we also evaluate models in pairs. These wrappers return the concatenation of the vectors from each model. (Only the singular models are trained.)

In [5]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

## Predictive Evaluation Methods

Helper methods for evaluating error rate.

In [3]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)


## Bulk Training

Using explicit multiple-pass, alpha-reduction approach as sketched in [gensim doc2vec blog post](http://radimrehurek.com/2014/12/doc2vec-tutorial/) – with added shuffling of corpus on each pass.

Note that vector training is occurring on *all* documents of the dataset, which includes all TRAIN/TEST/DEV docs.

Evaluation of each model's sentiment-predictive power is repeated after each pass, as an error rate (lower is better), to see the rates-of-relative-improvement. The base numbers reuse the TRAIN and TEST vectors stored in the models for the logistic regression, while the _inferred_ results use newly-inferred TEST vectors. 

(On a 4-core 2.6Ghz Intel Core i7, these 20 passes training and evaluating 3 main models takes about an hour.)

In [4]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

In [5]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list,total_examples=100000, epochs=epoch)
            duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2020-04-12 00:15:50.340293
*0.499800 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12) 0.0s 0.4s
*0.488400 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)_inferred 0.0s 6.0s
*0.499800 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12) 0.0s 0.4s
*0.488000 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)_inferred 0.0s 1.9s
*0.499800 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12) 0.0s 0.4s
*0.512000 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)_inferred 0.0s 2.7s
completed pass 1 at alpha 0.025000
*0.414120 : 2 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12) 26.3s 0.8s
*0.251240 : 2 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12) 12.2s 0.4s
*0.264480 : 2 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12) 18.3s 0.4s
completed pass 2 at alpha 0.023800
*0.314880 : 3 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12) 49.3s 0.4s
*0.129600 : 3 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t12) 23.3s 0.4s
*0.192120 : 3 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12) 36.3s 0

## Achieved Sentiment-Prediction Accuracy

In [6]:
# print best error rates achieved
for rate, name in sorted((rate, name) for name, rate in best_error.items()):
    print("%f %s" % (rate, name))

0.103160 Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)
0.107600 Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)_inferred
0.128840 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)
0.153000 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)
0.171600 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)_inferred
0.180800 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)_inferred


In my testing, unlike the paper's report, DBOW performs best. Concatenating vectors from different models only offers a small predictive improvement. The best results I've seen are still just under 10% error rate, still a ways from the paper's 7.42%.


## Examining Results

### Are inferred vectors close to the precalculated ones?

In [7]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

for doc 59103...
Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12):
 [(59103, 0.7466343641281128), (84048, 0.5244645476341248), (35561, 0.4812113046646118)]
Doc2Vec(dbow,d100,n5,mc2,s0.001,t12):
 [(59103, 0.9532005190849304), (62285, 0.5763416290283203), (36914, 0.5581868886947632)]
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12):
 [(59103, 0.7127737998962402), (51851, 0.6521816253662109), (48467, 0.6454958915710449)]


(Yes, here the stored vector from 20 epochs of training is usually one of the closest to a freshly-inferred vector for the same words. Note the defaults for inference are very abbreviated – just 3 steps starting at a high alpha – and likely need tuning for other applications.)

### Do close documents seem more related than distant ones?

In [13]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))


TARGET (38986): «when i went to see this movie it was already a forced choice , as my original intent was sold out . what ensuited then was sheer terror , this movie is so bad i could hardly bear it . the story is not worth mention , a gay goalkeeper forms a gay soccer team to play against his old straight team who - on discovering his sexual orientation - gave him a hard time . loaded with unbearably old and overused clichés of gays , the thin plot matches perfectly the inane dialogues . . . it is absolutely astonishing that actors as dietmar bär or charly hübner waste their talent and time on such nonsense . 1/10»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d100,n5,mc2,s0.001,t12):

MOST (16769, 0.49240642786026): «american pie : beta house is sort of in limbo between genres . on the one hand , it's a comedy with no plot and few genuinely clever jokes . on the other hand , it's porno that's a tad too soft-core to actually turn on any viewers . essentially beta house is a collage 

(Somewhat, in terms of reviewer tone, movie genre, etc... the MOST cosine-similar docs usually seem more like the TARGET than the MEDIAN or LEAST.)

### Do the word vectors show useful similarities?

In [9]:
word_models = simple_models[:]

In [16]:
import random
from IPython.display import HTML
# pick a random word with a suitable number of occurences
while True:
    word = random.choice(word_models[0].wv.index2word)
    if word_models[0].wv.vocab[word].count > 10:
        break
# or uncomment below line, to just pick a word from the relevant domain:
#word = 'comedy/drama'
similars_per_model = [str(model.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in word_models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in word_models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, simple_models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'ott' (78 occurences)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


"Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t12)","Doc2Vec(dbow,d100,n5,mc2,s0.001,t12)","Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t12)"
"[('over-the-top', 0.6089898943901062), ('histrionic', 0.5998550653457642), ('absurd', 0.5788363218307495), ('hammy', 0.5662363767623901), ('banal', 0.5611444711685181), ('overdone', 0.553699791431427), ('cheesy', 0.551530122756958), ('exaggerated', 0.5491220951080322), ('nonsensical', 0.5457489490509033), ('outrageous', 0.544755220413208), ('amateurish', 0.5411003828048706), ('melodramatic', 0.5316254496574402), ('corny', 0.5312181711196899), ('ridiculous', 0.531184196472168), ('unconvincing', 0.5283622741699219), ('contrived', 0.5263482928276062), ('laughable', 0.5254546403884888), ('overwrought', 0.5192746520042419), ('trite', 0.5185801982879639), ('stiff', 0.5175385475158691)]","[('repercussion', 0.4127863645553589), ('preform', 0.4025324583053589), (""hilter's"", 0.38194864988327026), (""bix's"", 0.3777480125427246), ('fortier', 0.37674254179000854), ('conflate', 0.3704046607017517), ('family-less', 0.3702741265296936), ('blood-frosting', 0.36308619379997253), ('sit-com', 0.36305761337280273), ('frumpish', 0.3627118766307831), ('severs', 0.3617934584617615), ('crapiness', 0.3558298945426941), ('cooker', 0.35284870862960815), ('theorize', 0.3489653468132019), (""net'"", 0.3482373356819153), ('brenna', 0.34773606061935425), ('recreating', 0.3475387394428253), ('stupefied', 0.3467150330543518), ('senegal', 0.34607475996017456), ('vanna', 0.3453220725059509)]","[('over-the-top', 0.4830499589443207), ('naturalistic', 0.4666609764099121), ('ominous', 0.4453126788139343), ('scornful', 0.4357379078865051), ('permissible', 0.4337683916091919), ('comical', 0.4285312294960022), ('orange', 0.4279036521911621), ('cheeky', 0.4272797107696533), ('socking', 0.4185159206390381), ('stylised', 0.4166119694709778), ('unsettling', 0.41254761815071106), ('childish', 0.4097590148448944), ('elfen', 0.4059644341468811), ('comedically', 0.4056779742240906), ('damaging', 0.4040599465370178), (""scrat's"", 0.4039596617221832), ('ultra-cool', 0.4021921157836914), ('creaking', 0.40052059292793274), ('overacted', 0.39773380756378174), ('outlandish', 0.3973613381385803)]"


Do the DBOW words look meaningless? That's because the gensim DBOW model doesn't train word vectors – they remain at their random initialized values – unless you ask with the `dbow_words=1` initialization parameter. Concurrent word-training slows DBOW mode significantly, and offers little improvement (and sometimes a little worsening) of the error rate on this IMDB sentiment-prediction task. 

Words from DM models tend to show meaningfully similar words when there are many examples in the training data (as with 'plot' or 'actor'). (All DM modes inherently involve word vector training concurrent with doc vector training.)

### Are the word vectors from this dataset any good at analogies?

In [17]:
# assuming something like
# https://word2vec.googlecode.com/svn/trunk/questions-words.txt 
# is in local directory
# note: this takes many minutes
for model in word_models:
    sections = model.wv.accuracy('questions-words.txt')
    correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
    print('%s: %0.2f%% correct (%d of %d)' % (model, float(correct*100)/(correct+incorrect), correct, correct+incorrect))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).
  


FileNotFoundError: [Errno 2] No such file or directory: 'questions-words.txt'

Even though this is a tiny, domain-specific dataset, it shows some meager capability on the general word analogies – at least for the DM/concat and DM/mean models which actually train word vectors. (The untrained random-initialized words of the DBOW model of course fail miserably.)

## Slop

In [ ]:
This cell left intentionally erroneous. 

To mix the Google dataset (if locally available) into the word tests...

In [ ]:
from gensim.models import Word2Vec
w2v_g100b = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
w2v_g100b.compact_name = 'w2v_g100b'
word_models.append(w2v_g100b)

To get copious logging output from above steps...

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

To auto-reload python code while developing...

In [ ]:
%load_ext autoreload
%autoreload 2